## 📈 Addressing Drift

<strong>Outline:</strong>


1. <u>Blind approach</u>

2. <u>Drift monitoring</u>
  - Monitor percentage change in residuals
  - Statistical process control
  - ADWIN

3. <u>Model retraining</u>



In [70]:
import pandas as pd
import plotly.express as px
from sklearn.base import BaseEstimator, RegressorMixin
from skmultiflow.drift_detection.adwin import ADWIN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from math import sqrt
import numpy as np
import os
from sklearn.pipeline import Pipeline
from typing import List, Union
from sklearn.base import clone

### Strategy 1: Blind approach


<li>Resembles online learning, tailored for use in a business context; </P></li>

<li><em> Blind approach</em>: doesn't rely on drift detection. </P></li>

#### How does it work?

<li>Model retraining is triggered at regular intervals using all training data available; </li></P>

<li>If the potential model outperforms the model that's currently deployed in production, the new model is deployed; </li></P>

<li>Otherwise, the old model is kept in production.</li></P>


<center><img src="Interval_Retraining.jpeg"  width="600" height="400" />



### Strategy 2: Drift monitoring

In [71]:
# Import 5 drifting time series
cwd = os.getcwd()
drift_path = os.path.abspath(os.path.join(cwd, 'Drift_TS.csv'))
drift_df = pd.read_csv(drift_path)

In [88]:
drift_df.replace({'5338101002002AG11': '1', '1406020043002AG11': '2', '1700111001001AG11': '3', '5130200014002AG11': '4', '5338101002002AG15': '5'}, inplace=True)
csv_path = cwd + '/Drift_TS.csv'
drift_df.to_csv(csv_path)

In [73]:
# Safely disable safely the chained assignment warning assignment
pd.options.mode.chained_assignment = None

In [74]:
# Define features and target variable
drift_df.set_index('LoadDate', inplace=True)
X = drift_df.drop(['OrderQtyFirst', 'Unnamed: 0'] , axis=1)
y = drift_df[['OrderQtyFirst']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
test = pd.concat([X_test["Group_Item"], y_test], axis=1)

In [75]:
# Define pipeline
standard_scaler = ColumnTransformer([('standard_scaler', StandardScaler(), make_column_selector(dtype_include=int))], remainder=OneHotEncoder())
model = RandomForestRegressor()
pipeline = Pipeline([('preprocessor', standard_scaler),('regressor', model)])

In [76]:
class MetaModel(BaseEstimator, RegressorMixin):

    def __init__(self, basepipeline: Pipeline, itemGroupColumn: str):
        self.basepipeline = basepipeline
        self.itemGroupColumn = itemGroupColumn
        self.modelDict = {}

    def fit(self, X: np.array, y: np.array) -> None:
        """Fit a model per itemGroup.

        Args:
            X (np.array): Training data to fit on.
            y (np.array): Corresponding target variable.
        """
        for itemGroup in X[self.itemGroupColumn].unique():
            ids = X[self.itemGroupColumn] == itemGroup
            xItemGroup = X.loc[ids]
            yItemGroup = y[ids]
            itemGroupModel = clone(self.basepipeline)
            itemGroupModel.fit(xItemGroup, yItemGroup.values.ravel())
            self.modelDict[itemGroup] = itemGroupModel

    def predict(self, X: np.array, y: np.array) -> pd.DataFrame:
        """Apply predictions per itemGroup model and calculate residuals.

        Args:
            X (np.array): Testing data to make predictions for.
            y (np.array): Target variable for testing data used to calculate residuals.

        Returns:
            itemGroupPrediction (np.array):  Predictions for each itemGroup.
            residuals (np.array): Residuals for each itemGroup.

        """
        PredictionsList = []
        for itemGroup in X[self.itemGroupColumn].unique():
            model = self.modelDict[itemGroup]
            ids = X[self.itemGroupColumn] == itemGroup
            xItemGroup = X.loc[ids]
            itemGroupPrediction = model.predict(xItemGroup)
            xItemGroup['Prediction'] = itemGroupPrediction
            PredictionsList.append(xItemGroup)
        df = pd.concat(PredictionsList)
        return df


In [77]:
meta_model = MetaModel(basepipeline=pipeline, itemGroupColumn='Group_Item')
meta_model.fit(X_train, y_train)
df = meta_model.predict(X_test, y_test)
df = test.merge(df, on=['LoadDate', 'Group_Item'], how='inner')

In [78]:
df['Residuals'] = df['OrderQtyFirst'] - df['Prediction']

### Strategy 2.1: Track percentage change between residuals

#### How does it work?

<li>Percentage change in residuals is calculated for a set of days per item group;</li></P>

<li>Applied for the change between today's residuals and previous dates;</li></P>

<li>Raise an alarm for after x number of days.</li></P>


In [79]:
def calculate_percentage_change(dataframe: pd.DataFrame, itemGroupColumn: str, residualsColumn: str, difference: List  = [1,7,14,30,100]) -> pd.DataFrame:
    """Calculate percentage change in residuals for each itemGroup.

    Args:

        dataframe (pd.DataFrame): Dataframe populated with item group and residuals.

        itemGroupColumn (str): Column name for itemGroup.   

        difference (List, optional): List of days to calculate percentage change for between. Defaults to [1,7,14,30,100].

    Returns:
        pd.DataFrame: Dataframe populated with percentage change for residuals.
    """
    result = []
    for itemGroup in dataframe[itemGroupColumn].unique():
        ids = dataframe[itemGroupColumn] == itemGroup
        item_df = dataframe.loc[ids]
        for diff in difference:
            item_df[f'{diff}_day_change'] = np.abs(item_df[residualsColumn].shift(diff) / item_df[residualsColumn]) * 100
        item_df = item_df[item_df['100_day_change'].notna()]
        result.append(item_df)
    return pd.concat(result)


In [80]:
results = calculate_percentage_change(df, 'Group_Item', 'Residuals')
results.drop(['Year', 'Quarter', 'Month', 'Week', 'DayOfWeek', 'DayOfYear'], axis=1, inplace=True)
results = results.groupby(['Group_Item']).tail(1)
print(results)

                          Group_Item  OrderQtyFirst  Prediction  Residuals  \
LoadDate                                                                     
2021-11-24 00:00:00+00:00          1           10.0       10.37      -0.37   
2021-11-24 00:00:00+00:00          2            6.0        6.14      -0.14   
2021-11-24 00:00:00+00:00          3           28.0       18.50       9.50   
2021-11-24 00:00:00+00:00          4           14.0        5.83       8.17   
2021-11-23 00:00:00+00:00          5            4.0        6.22      -2.22   

                           1_day_change  7_day_change  14_day_change  \
LoadDate                                                               
2021-11-24 00:00:00+00:00   2770.270270   1810.810811    1472.972973   
2021-11-24 00:00:00+00:00   2907.142857   4814.285714    1064.285714   
2021-11-24 00:00:00+00:00    103.684211    187.157895     202.631579   
2021-11-24 00:00:00+00:00      5.630355     69.767442      23.867809   
2021-11-23 00:00:00+0

### Strategy 2.2: Statistical process control (SPC)

SPC is a method of quality control which is commonly used in manufacturing or production processes to monitor and control a process.
It can be applied to track either the target variable or the residuals.

### How does it work?

An <em>upper</em> and a <em>lower bound</em> are computed using the mean and standard deviation. Drift is detected when x points in the span of any y days fall oustide the upper or lower bound.

### Note:

Residuals from the testing set should be used as a baseline when compared against residuals of a deployed model.

<strong>Reference:</strong> [Control chart based on residues: Is a good methodology to detect outliers?](https://link.springer.com/article/10.1007/s40092-019-00324-0)

In [81]:
class SPC():

    def __init__(self, delta: int, window:int = 20, minPoints:int = 10):
        self.delta = delta
        self.window = window
        self.minPoints = minPoints
        self.pointList = []
        self.sigma = 0
        self.upperBound = 0
        self.lowerBound = 0
        self.processMean = 0


    def calculate_stats(self, datastream: np.array) -> None:
        """Calculate the mean, standard deviation, and bounds of the datastream.
        
        Args:
            datastream (np.array): Datastream to calculate stats for."""
            
        self.processMean = datastream.mean()
        self.sigma = datastream.std()
        self.upperBound = self.processMean + self.delta * self.sigma
        self.lowerBound = self.processMean - self.delta * self.sigma

    def add_element(self, datastream: np.array) -> List:
        """Test each datapoint inside datastream against upper and lower bound and store results in pointList. 
        False refers to no drift and True refers to drift.

        Args:
            datastream (np.array): Data to be tested. Should be a 1D array.

        Returns:
            pointList (List): List populated with booleans based on testing condition for each point in datastream.
        """
        self.calculate_stats(datastream)
        self.pointList = np.logical_or(datastream <= self.lowerBound, datastream >= self.upperBound)
        return self.pointList

    def detected_change(self) -> bool:
        """Test whether a certain number of points fall outside upper and lower bound within a certain number of days.

        Returns:
            : bool: True if drift is detected, False if not.
        """
        num_errors = np.sum(np.lib.stride_tricks.sliding_window_view(self.pointList, window_shape=(self.window,)), axis=1)
        drift_comparasion = num_errors >= self.minPoints
        drift_region= np.nonzero(drift_comparasion) 
        return drift_comparasion, drift_region

    def plotSPC(self, datastream):
        fig = px.line(datastream)
        fig.update_xaxes(title_text='Date')
        fig.update_yaxes(title_text='Residuals')
        drift_region = self.detected_change()[1]
        fig.update_layout(title=f'Process Control Chart | Mean: {int(self.processMean)}, SD: {int(self.sigma)}', showlegend=False, title_x=0.5)
        fig.update_traces(line_color='#ff7f0e', line_width=1)
        fig.add_hline(y=self.processMean, line_color="black", line_width=1, line_dash="dash")
        fig.add_hline(y=self.lowerBound, line_color="red", line_width=1, annotation_text=f"<em>Lower bound</em>: {int(self.lowerBound)}", annotation_position="bottom right")
        fig.add_hline(y=self.upperBound, line_color="red", line_width=1, annotation_text=f"<em>Upper bound</em>: {int(self.upperBound)}")
        fig.show()

In [82]:
# plot order qty first per item group
fig = px.line(drift_df, x=drift_df.index, y="OrderQtyFirst", color="Group_Item")
fig.update_traces(line_width=1)
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Quantity')
fig.show()

In [83]:
for itemGroup in df['Group_Item'].unique():
    spc = SPC(delta=2, window=30, minPoints=5)
    ids = df['Group_Item'] == itemGroup
    df_temp = df.loc[ids]
    spc.add_element(df_temp['Residuals'])
    print('Item Group:', itemGroup)
    print('Window Indices:', spc.detected_change()[1]) 
    spc.plotSPC(df_temp['Residuals']) 

Item Group: 1
Window Indices: (array([], dtype=int64),)


Item Group: 2
Window Indices: (array([316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328,
       329, 330, 331, 332, 333, 334, 335]),)


Item Group: 3
Window Indices: (array([290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302,
       303, 304, 305, 306, 307, 308, 309, 310, 311, 312]),)


Item Group: 4
Window Indices: (array([327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
       340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352,
       353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365,
       366, 367, 368, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416,
       417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428]),)


Item Group: 5
Window Indices: (array([137, 138, 139, 140, 141, 572, 573, 574, 575, 576, 577, 578, 579,
       580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591]),)


###  Strategy 2.3: ADaptive WINdowing (ADWIN):
ADWIN is a drift detection algorithm which uses sliding windows whose
size is computed online according to the rate of change observed from the data inside the window.



### How does it work?

A window W of <em>variable</em> size is maintained and whenever two large enough subwindows of W exhibit distinct enough averages, drift is found and the older portion of the window is dropped. Therefore, the algorithm automatically grows the window when no change is found, and shrinks it when data changes.

### Note on parameters: 

Confidence value is the only paramater which ADWIN requires and it represents a false positive rate.

<li>If the mean remains constant
within W, the probability that ADWIN shrinks the
window at this step is at most δ.</P></li>

There's a tradeoff between this parameter and the false negative rate

### Inputs:

<ul>
<li>Confidence value (False positive rate bound) δ ∈ (0, 1)</P>
<ul>
      <li>If the mean remains constant
within W, the probability that ADWIN shrinks the
window at this step is at most δ.</P></li>
</ul>
 </li>
<li>Sequence of real values. </P></li>
</ul>

<strong>Reference:</strong> [Bifet and R. Gavalda. 2007. Learning from Time-Changing Data with Adaptive Windowing](https://epubs.siam.org/doi/pdf/10.1137/1.9781611972771.42)

In [84]:
class DriftDetector():

    def __init__(self, baseDetector):
        self.baseDetector = baseDetector
        self.changepoints = {}

    
    def find_drift(self, df: pd.DataFrame, itemgroup_column: str, datastream_column: str) -> dict:
        """Use drift detector to find changepoints in datastream.

        Args:
            df (pd.DataFrame): Takes in a dataframe with itemgroup and datastream columns. Datastream represnts the datastream to be tested for drift.
            itemgroup_column (str): Column name for itemgroup.
            datastream_column (str): Column name for datastream.

        Returns:
            dict: Dictionary of detected changepoints per item group.
        """
        for item_group in df[itemgroup_column].unique():
            points = []
            ids = df[itemgroup_column] == item_group
            datastream = np.array(df.loc[ids][datastream_column])
            for i in range(len(datastream)):    
                self.baseDetector.add_element(datastream[i])
                if self.baseDetector.detected_change():
                    points.append(i)
            self.changepoints[item_group] = points
        return self.changepoints

In [85]:
drift = DriftDetector(ADWIN(delta=0.005))
drift.find_drift(df, 'Group_Item', 'Residuals')

{'1': [],
 '2': [16],
 '3': [298, 330, 362, 554],
 '4': [321, 385, 513],
 '5': [194, 258, 290, 322, 418, 514, 578]}

### Model Retraining
<center><img src="Model_Retraining.jpg" width="400" height="200" />
